In [7]:
import os
import pandas as pd
import numpy as np
import plotly.express as px

In [53]:
data = pd.read_excel(io="../data/ine/ajuste_estacional_historico.xlsx", sheet_name="tasa_as")
data = data.loc[3:172]
data.columns= pd.MultiIndex.from_arrays(data.iloc[0:2].to_numpy())
data = data.loc[6:].reset_index(drop=True)
data.set_index([("Año", np.nan), ("Trimestre", np.nan)], inplace=True)
data.index.names = ["Año", "Trimestre"]
data.reset_index(["Año", "Trimestre"], inplace=True)
data["Año, Trimestre"] = data[["Año", "Trimestre"]].aggregate(lambda x: ", ".join(x.astype(str)), axis=1)
df = data.copy()
data.columns = pd.Index(map(str.strip, data.columns.get_level_values(0) + " " + data.columns.get_level_values(1)))

fig = px.line(data, x="Año, Trimestre", y=["Tasa oficial DEF2020", "Tasa ajustada DEF2024", "Tasa oficial DEF2024", "Tasa ajustada DEF2024"], 
              markers=True)
fig

C:\Users\aurib\AppData\Local\Temp\ipykernel_24032\1846587850.py:7: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\aurib\AppData\Local\Temp\ipykernel_24032\1846587850.py:7: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\aurib\AppData\Local\Temp\ipykernel_24032\1846587850.py:8: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

Comprobamos si los datos oficiales efectivamente vienen de la encuesta ENE o de la preicción hecha por el modelo de la misma

In [54]:
origin = pd.concat(
    [df.loc[:, ["Tasa oficial"]].aggregate(func=lambda row: len(row.value_counts())<=1, axis=1), 
     df["Año, Trimestre"]], axis=1) \
        .rename({0:"valores iguales en cada modelo"}, axis=1)
more = origin["Año, Trimestre"].loc[~origin["valores iguales en cada modelo"]].index
origin

,valores iguales en cada modelo,"Año, Trimestre"
0,True,"2024, May - Jul"
1,True,"2024, Abr - Jun"
2,True,"2024, Mar - May"
3,True,"2024, Feb - Abr"
4,True,"2024, Ene - Mar"
...,...,...
162,False,"2010, Nov - Ene"
163,False,"2010, Oct - Dic"
164,False,"2010, Sep - Nov"
165,False,"2010, Ago - Oct"


In [56]:
for index in more:
    print(df.loc[index,"Tasa oficial"].value_counts())

24
7.918812    24
7.918812     1
Name: count, dtype: int64
25
7.811474    25
7.811474     1
Name: count, dtype: int64
26
7.80398    26
7.80398     1
Name: count, dtype: int64
27
7.747493    27
7.747493     1
Name: count, dtype: int64
28
7.797865    28
7.797865     1
Name: count, dtype: int64
29
7.498522    29
7.498522     1
Name: count, dtype: int64
30
7.271465    30
7.271465     1
Name: count, dtype: int64
31
7.189893    31
7.189893     1
Name: count, dtype: int64
32
7.527649    32
7.527649     1
Name: count, dtype: int64
34
8.405611    34
8.405611     1
Name: count, dtype: int64
35
8.547566    35
8.547566     1
Name: count, dtype: int64
36
8.933289    36
8.933289     1
Name: count, dtype: int64
37
9.496488    37
9.496488     1
Name: count, dtype: int64
38
10.037301    38
10.037301     1
Name: count, dtype: int64
39
10.244367    37
10.244367     2
10.244367     1
Name: count, dtype: int64
40
10.35381    37
10.35381     4
Name: count, dtype: int64
41
10.304521    37
10.304521     5
Nam

Podemos observar que los valores utilizados en los modelos pueden presentar ciertas varaciones por trimestre movil. Esa variación parece ser de orden despreciable; $10^6$ como mínimo (exceptuando algunos trimestres). Desde aquí, podemos proceder a utilizar la tasa oficial desde a encuentra ENE procesada por este equipo. **Preguntar a INE**